#  Embeddings & VectorStores

This notebook show cases how to use embeddings to create a VectorStore

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores.faiss import FAISS

In [2]:
with open('../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [3]:
docsearch = FAISS.from_texts(texts, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

In [4]:
import numpy as np
embedding = docsearch.embedding_function(query)
embs, indices = docsearch.index.search(np.array([embedding], dtype=np.float32), 20)


In [30]:
cosine_similarity(embedding, embeddings[3])

0.7665002653224218

In [51]:

import numpy as np

def cosine_similarity(a,b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

def maximal_marginal_relevance(sentence_vector, embedding_list, lambda_constant=0.5, threshold_terms=4):
    """"""
    # TODO: Use cosine similarity matrix for lookup among phrases instead of making call everytime.
    idxs = []
    while len(idxs) < threshold_terms:
        best_score = -np.inf
        idx_to_add = -1
        for i, emb in enumerate(embedding_list):
            if i in idxs:
                continue
            first_part = cosine_similarity(sentence_vector, emb)
            second_part = 0
            for j in idxs:
                cos_sim = cosine_similarity(emb, embedding_list[j])
                if cos_sim > second_part:
                    second_part = cos_sim
            equation_score = lambda_constant*(first_part)-(1-lambda_constant) * second_part
            if equation_score > best_score:
                best_score = equation_score
                idx_to_add = i
        idxs.append(idx_to_add)
    return idxs

In [52]:
embeddings = [docsearch.index.reconstruct(int(i)) for i in indices[0]]

In [53]:
maximal_marginal_relevance(embedding, embeddings)

[0, 2, 10, 3]

In [4]:
print(docs[0].page_content)

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the

## Requires having ElasticSearch setup

In [6]:
docsearch = ElasticVectorSearch.from_texts(texts, embeddings, elasticsearch_url="http://localhost:9200")

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

In [7]:
print(docs[0].page_content)

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the